In [ ]:
import numpy as np
import pandas as pd
import time
import librosa
import librosa.display

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

from boris_util import *

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN

from sklearn.pipeline import Pipeline

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Exploring the Dataset

First thing's first, let's load the dataset. I created a new function that loads up the dataset for me and converts it into a pandas dataframe for easy access. For more information on this, please see the `Data Cleaning` notebook

In [ ]:
dataset = read_and_combine_data('453_923_bundle_archive', read_noisy_data=False)
print(dataset.shape)
dataset.head()

Let's take a look at the class distribution to see if there is any concern of class imbalance

In [ ]:
label_counts = dataset['label'].value_counts()

plt.figure(figsize=(15,5))
plt.barh(label_counts.index, label_counts.values)
plt.xlabel("Value Counts")
plt.ylabel("Condition")
plt.title("The Distribution of Different Labels in the Dataset")
plt.show()

There is certainly an issue of class imbalance here. A naive classifier could guess pretty well by simply guessing "Normal" on all the examples

## Combine all Abnormalities

Let's try a simple example first, we will combine all abnormalities into a single label "abnormal". We won't count artifacts in that category

In [ ]:
def relabel(label):
    if (label == "normal"):
        return "normal"
    else:
        return "abnormal"
dataset['label'] = dataset['label'].apply(relabel)

In [ ]:
label_counts = dataset['label'].value_counts()

plt.figure(figsize=(15,5))
plt.barh(label_counts.index, label_counts.values)
plt.xlabel("Value Counts")
plt.ylabel("Condition")
plt.title("The Distribution of Different Labels in the Re-labeled Dataset")
plt.show()

Let's split our data now into a simple train/test stratified split

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, stratify=dataset['label'], random_state=1)
print(train_dataset.shape)
print(test_dataset.shape)

In [ ]:
train_dataset.head()

## Dataset Augmentation

We won't get very far with the examples we have. There aren't very many of them

Luckily, we can easily augment the dataset by shifting the data and/or adding some noise

In [ ]:
DATASET_TARGET = 1000

In [ ]:
# find the label ratios
inverse_counts = 1/train_dataset['label'].value_counts()
probability_counts = inverse_counts/inverse_counts.sum()
probability_counts

In [ ]:
#https://medium.com/@makcedward/data-augmentation-for-audio-76912b01fdf6
    
def shift_sound(data, sampling_rate, shift_max, shift_direction='both'):
    shift = np.random.randint(int(shift_max * data.shape[0]))
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data = augmented_data[shift:]
    else:
        augmented_data = augmented_data[:shift]
    return augmented_data

In [ ]:
choices = np.random.choice(probability_counts.index, 
                           p=probability_counts.values, 
                           size=(DATASET_TARGET,))
i = 0
while train_dataset.shape[0] < DATASET_TARGET:
    choice = choices[i]
    available_instances = train_dataset[train_dataset['label'] == choice]
    selected_row_position = np.random.randint(available_instances.shape[0])
    
    selected_row = available_instances.iloc[selected_row_position]
    
    base_signal = selected_row['signal']
    sampling_rate = selected_row['sampling_rate']
    
    shift_direction = 'both'
    new_signal = shift_sound(base_signal, sampling_rate, (np.random.random()*0.3), shift_direction)
    if (new_signal.shape[0] > 0):
    
        new_series = pd.Series({'signal' :new_signal, 'sampling_rate': sampling_rate, 'label': choice})
        train_dataset = train_dataset.append(new_series, ignore_index=True)
    
        print(train_dataset.shape, end="\r")
    i+= 1

Now let's define a class which will convert the signals into Mel spectrograms

In [ ]:
class MelEncoder:
    
    '''
        This transformer takes an audio signal and employes librosa
        to transform the audio signal into a Mel-frequency spectrogram.
        
        The transformation transforms the audio signal from samples in a 
        time domain, to frequencies over each time window. The frequencies
        are then collected into buckets which are easily distinguished by
        human hearing
    '''
    
    def __init__(self, flatten=True, n_fft=2048, hop_length=512, n_mels=128, sampling_rate=22050.0, n_windows=1000, verbose=0):
        
        
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.sampling_rate = sampling_rate
        self.n_windows = n_windows
        self.verbose = verbose
        
        
        self.flatten = flatten
        
    
    def fit(self, X, y=None):
        '''
            This method doesn't do much except initialize
        '''
        self.longest_signal_length = None
        
    
    
    def _create_spectrogram_list(self, X):
        
        spectrogram_list = []
        
        
        for i, signal in enumerate(X):
                mel_spectrogram = self._signal_to_mel_spectrogram(signal)
                spectrogram_list.append(mel_spectrogram)
                if (self.verbose):
                    print(f"processed: {round(100*i/X.shape[0], 2)}%", end="\r")
                    
                
        print(f"processed: {round(100*i/X.shape[0], 2)}%")
        
        return spectrogram_list
    
    
    def _find_longest_signal_length(self, spectrogram_list):
        
        max_length = 0
        for mel_spectrogram in spectrogram_list:
            if (mel_spectrogram.shape[1] > max_length):
                max_length = mel_spectrogram.shape[1]

        return max_length
            
    def _build_mel_spectrogram_stack(self, spectrogram_list):
        
        X_transformed = np.zeros((len(spectrogram_list), self.n_mels, self.longest_signal_length))
        
        for i, mel_spectrogram in enumerate(spectrogram_list):
            if (mel_spectrogram.shape[1] <= self.longest_signal_length):
                X_transformed[i, :, -mel_spectrogram.shape[1]:] = mel_spectrogram[:, :]
            else:
                X_transformed[i, :, :] = mel_spectrogram[:, -self.longest_signal_length:]
                
        return X_transformed
    
    
    def transform(self, X):
        
        spectrogram_list = self._create_spectrogram_list(X)
        
        if (self.longest_signal_length is None):
            self.longest_signal_length = self._find_longest_signal_length(spectrogram_list)
            
        X_transformed = self._build_mel_spectrogram_stack(spectrogram_list)
        
        
        if (self.flatten):
            return X_transformed.reshape(X_transformed.shape[0], -1)
        else:
            return X_transformed
        
        
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)
        
        
        
        
    # https://towardsdatascience.com/getting-to-know-the-mel-spectrogram-31bca3e2d9d0
    def _longest_signal(self, signals):
        max_length = 0
        for signal in signals:
            if (len(signal) > max_length):
                max_length = len(signal)

        return max_length
    
    
    def _signal_to_mel_spectrogram(self, signal):
        
        
        if (self.n_windows is not None):
            hop_length = signal.shape[0]//self.n_windows
        else:
            hop_length = self.hop_length
        S = librosa.feature.melspectrogram(signal, 
                                           sr=self.sampling_rate, 
                                           n_fft=self.n_fft, 
                                           hop_length=hop_length, 
                                           n_mels=self.n_mels)
        S_DB = librosa.power_to_db(S, ref=np.max)
        return S_DB

In [ ]:
my_encoder = MelEncoder(n_mels=128, n_windows=1000, verbose=1)

In [ ]:
my_encoder.fit(train_dataset['signal'])

In [ ]:
X_train_mel = my_encoder.transform(train_dataset['signal'])

In [ ]:
X_train_mel.shape

In [ ]:
X_test_mel = my_encoder.transform(test_dataset['signal'])

In [ ]:
X_test_mel.shape

## PCA

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
my_mmscaler = MinMaxScaler()

X_train_mel = my_mmscaler.fit_transform(X_train_mel)
X_test_mel = my_mmscaler.transform(X_test_mel)

In [ ]:
X_train_mel = X_train_mel
X_train_mel.shape

In [ ]:
%%time 

component_number = min(*X_train_mel.shape)
my_pca = PCA(n_components=component_number, svd_solver='full')
my_pca.fit(X_train_mel)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(my_pca.explained_variance_ratio_.cumsum())
plt.show()

In [ ]:
X_train_mel = my_pca.transform(X_train_mel)
X_test_mel = my_pca.transform(X_test_mel)

In [ ]:
X_train_mel.shape

In [ ]:
X_test_mel.shape

## Basic Model - Logistic Regression

Let's start by establishing a baseline performance. I am just going to load the raw audio data as features (where each feature is an audio measurement) and see how well a simple logistic regression (linear model) and decision tree (non-linear model) will perform

In [ ]:
condition_label_encoder = LabelEncoder()


y_train = condition_label_encoder.fit_transform(train_dataset['label'])[:X_train_mel.shape[0]]
y_test = condition_label_encoder.transform(test_dataset['label'])

print(X_train_mel.shape, X_test_mel.shape)
print(y_train.shape, y_test.shape)

In [ ]:
%%time

my_logit = LogisticRegression(random_state=1).fit(X_train_mel, y_train)
train_acc = my_logit.score(X_train_mel, y_train)
test_acc = my_logit.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

Extremely suspicious results. There is clearly a very large sign of overfitting here. Let's explore how the logistic regression makes decisions.

Let's plot the coefficients as if they were an audio mask (which is really what they end up becoming).

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_logit, X_test_mel, y_test,
                             display_labels = condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize=None,
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_logit, X_test_mel, y_test,
                             display_labels=condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize='true',
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
param_grid = {'penalty': ['l1', 'l2'],
              'C': [2**i for i in range(-17, 18)]}
my_estimator = LogisticRegression(solver='saga')
my_logit_gs = GridSearchCV(my_estimator, param_grid, cv=5, verbose=1, n_jobs=3)

In [ ]:
%%time
my_logit_gs.fit(X_train_mel, y_train);

In [ ]:
my_logit_gs.best_params_

In [ ]:
train_acc = my_logit_gs.score(X_train_mel, y_train)
test_acc = my_logit_gs.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_logit_gs, X_test_mel, y_test,
                             display_labels = condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize=None,
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_logit_gs, X_test_mel, y_test,
                             display_labels=condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize='true',
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
print(classification_report(y_test, my_logit_gs.predict(X_test_mel)))

In [ ]:
y_proba = my_logit_gs.predict_proba(X_test_mel)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_proba[:, 1], y_test)
plt.show();

## Basic Model - Support Vector Machine

Let's start by establishing a baseline performance. I am just going to load the raw audio data as features (where each feature is an audio measurement) and see how well a simple logistic regression (linear model) and decision tree (non-linear model) will perform

In [ ]:
condition_label_encoder = LabelEncoder()


y_train = condition_label_encoder.fit_transform(train_dataset['label'])[:X_train_mel.shape[0]]
y_test = condition_label_encoder.transform(test_dataset['label'])

print(X_train_mel.shape, X_test_mel.shape)
print(y_train.shape, y_test.shape)

In [ ]:
%%time

my_svm = SVC(random_state=1, max_iter=10000, probability=True).fit(X_train_mel, y_train)
train_acc = my_svm.score(X_train_mel, y_train)
test_acc = my_svm.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

In [ ]:
param_grid = {'C': [2**i for i in range(-17, 18)],
              'kernel': ['poly', 'rbf', 'sigmoid']}
my_estimator = SVC(max_iter=10000, probability=True)
my_svm_gs = GridSearchCV(my_estimator, param_grid, cv=5, verbose=1, n_jobs=3)

In [ ]:
%%time
my_svm_gs.fit(X_train_mel, y_train);

In [ ]:
my_svm_gs.best_params_

In [ ]:
train_acc = my_svm_gs.score(X_train_mel, y_train)
test_acc = my_svm_gs.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_svm_gs, X_test_mel, y_test,
                             display_labels = condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize=None,
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_svm_gs, X_test_mel, y_test,
                             display_labels=condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize='true',
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
print(classification_report(y_test, my_svm_gs.predict(X_test_mel)))

In [ ]:
y_proba = my_svm_gs.predict_proba(X_test_mel)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_proba[:, 1], y_test)
plt.show();

## Basic Model - KNN

Let's start by establishing a baseline performance. I am just going to load the raw audio data as features (where each feature is an audio measurement) and see how well a simple logistic regression (linear model) and decision tree (non-linear model) will perform

In [ ]:
condition_label_encoder = LabelEncoder()


y_train = condition_label_encoder.fit_transform(train_dataset['label'])[:X_train_mel.shape[0]]
y_test = condition_label_encoder.transform(test_dataset['label'])

print(X_train_mel.shape, X_test_mel.shape)
print(y_train.shape, y_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [ ]:
%%time

my_knn = KNN().fit(X_train_mel, y_train)
train_acc = my_knn.score(X_train_mel, y_train)
test_acc = my_knn.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

Extremely suspicious results. There is clearly a very large sign of overfitting here. Let's explore how the logistic regression makes decisions.

Let's plot the coefficients as if they were an audio mask (which is really what they end up becoming).

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_knn, X_test_mel, y_test,
                             display_labels = condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize=None,
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_knn, X_test_mel, y_test,
                             display_labels=condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize='true',
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
param_grid = {'knn__n_neighbors': [i for i in range(3,10)] + \
                             [i for i in range(10, 100, 5)] + \
                             [i for i in range(100, X_train_mel.shape[0]//2, 20)],
              'knn__weights': ['uniform', 'distance']}

my_pipe = Pipeline([('scaler', MinMaxScaler()),
                    ('knn', KNN(algorithm='auto'))])

my_knn_gs = GridSearchCV(my_pipe, param_grid, cv=10, verbose=1, n_jobs=3)

In [ ]:
%%time
my_knn_gs.fit(X_train_mel, y_train);

In [ ]:
my_knn_gs.best_params_

In [ ]:
train_acc = my_knn_gs.score(X_train_mel, y_train)
test_acc = my_knn_gs.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_knn_gs, X_test_mel, y_test,
                             display_labels = condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize=None,
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_knn_gs, X_test_mel, y_test,
                             display_labels=condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize='true',
                             ax=ax)
disp.ax_.set_title("Logit Confusion Matrix")
plt.show()

In [ ]:
print(classification_report(y_test, my_knn_gs.predict(X_test_mel)))

In [ ]:
y_proba = my_knn_gs.predict_proba(X_test_mel)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_proba[:, 1], y_test)
plt.show();

## Basic Model - Decision Tree

As another baseline, I will train a decision tree to see if introducing non-linearity to the learning process produces better results

In [ ]:
print(X_train_mel.shape, X_test_mel.shape)
print(y_train.shape, y_test.shape)

In [ ]:
%%time 

# Let's set some regularization otherwise we know the tree will overfit
my_tree = DecisionTreeClassifier()
param_grid = {'min_samples_leaf': [i for i in range(1, 20)],
              'max_depth' : [i for i in range(1, X_train_mel.shape[1], X_train_mel.shape[1]//10)]+[None],
              'max_features': [i/10 for i in range(2, 11, 2)]}

my_tree_gs = GridSearchCV(my_tree, param_grid, cv=5, verbose=1, n_jobs=3)
my_tree_gs.fit(X_train_mel, y_train)


train_acc = my_tree_gs.score(X_train_mel, y_train)
test_acc = my_tree_gs.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

In [ ]:
my_tree_gs.best_params_

Much better results! However, let's see how the model performs using the confusion matrix of the test set



In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_tree_gs, X_test_mel, y_test,
                             display_labels = condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize=None,
                             ax=ax)
disp.ax_.set_title("Decision Tree Confusion Matrix")
plt.show()

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_tree_gs, X_test_mel, y_test,
                             display_labels=condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize='true',
                             ax=ax)
disp.ax_.set_title("Decision Tree Confusion Matrix")
plt.show()

As suspected, quite a bit of the classification seems to classify abnormal conditions and sounds as "normal" due to the sheer volume of that class, but interestingly, ~39% of the normal examples get misclassified as abnormalities.

In [ ]:
print(classification_report(y_test, my_tree_gs.predict(X_test_mel)))

In [ ]:
y_proba = my_tree_gs.predict_proba(X_test_mel)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_proba[:, 1], 
            y_test)
plt.show();

## Advanced Model - Random Forest

...

In [ ]:
print(X_train_mel.shape, X_test_mel.shape)
print(y_train.shape, y_test.shape)

In [ ]:
%%time 

# Let's set some regularization otherwise we know the tree will overfit
my_rf = RandomForestClassifier(min_samples_leaf=10, 
                               max_features=0.6,
                               max_depth=901,
                               random_state=1, 
                               n_estimators=350,
                               verbose=1,
                               n_jobs=3).fit(X_train_mel, y_train)


train_acc = my_rf.score(X_train_mel, y_train)
test_acc = my_rf.score(X_test_mel, y_test)

In [ ]:
print(f"The training accuracy is {round(train_acc*100,2)}%")
print(f"The test accuracy is {round(test_acc*100,2)}%")

Much better results! However, let's see how the model performs using the confusion matrix of the test set



In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_rf, X_test_mel, y_test,
                             display_labels = condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize=None,
                             ax=ax)
disp.ax_.set_title("Gradient Boosting Confusion Matrix")
plt.show()

In [ ]:
figure, ax = plt.subplots(figsize=(7, 7))
disp = plot_confusion_matrix(my_rf, X_test_mel, y_test,
                             display_labels=condition_label_encoder.classes_,
                             cmap=plt.cm.Blues,
                             normalize='true',
                             ax=ax)
disp.ax_.set_title("Gradient Boosting Confusion Matrix")
plt.show()

In [ ]:
print(classification_report(y_test, my_rf.predict(X_test_mel)))

In [ ]:
y_proba = my_rf.predict_proba(X_test_mel)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_proba[:, 1], 
            y_test)
plt.show();